<a href="https://colab.research.google.com/github/crapptrapp/sadGPT/blob/main/sadGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using https://github.com/agrechnev/hugging_examples/blob/master/train_gpt2_torch1.py

In [1]:
!pip install git+https://github.com/huggingface/transformers wandb pandas torch 'transformers[torch]'

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-w_lxb8jc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-w_lxb8jc
  Resolved https://github.com/huggingface/transformers to commit a6e6b1c622d8d08e2510a82cb6266d7b654f1cbf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
def load_data(URL):
  quotes=pd.read_csv(URL,header=None)
  quotes=quotes.fillna('')
  return quotes
quotes=load_data("https://huggingface.co/datasets/Crapp/sadQuotes/raw/main/quotes.csv")

In [65]:
quote=quotes.iloc[:][1]
quote=quote.str.split('.').explode()
type(quote)
quote="\n".join(quote)

In [67]:
str2file = open("quotes.txt", "w")
str2file.write(quote)
str2file.close()
#quote.to_csv(r'quotes.txt', header=None, index=None, sep=' ', mode='w')

In [68]:
!head quotes.txt

What is happening in your Mind is not Reality; it is important to differentiate between the two

Once you cultivate Equanimity within, every cell in your body will respond by generating Sweetness

Life is a limited amount of Time and Energy
 Let us use it for maximum Impact

What is good for the soil is always good for your body because your body is just an embodiment of soil

Our intention is to make this Planet into a Temple where everyone walks with a certain Grace and reverence to Life


In [69]:
import sys
import numpy as np
import torch
import torch.utils.data
import transformers
import tqdm
MODEL_NAME = 'gpt2'
TEXT_CORPUS = 'quotes.txt'
DEVICE = 'cuda'
TOKEN_ENDOFTEXT = 50256  # '<|endoftext|>
BLOCK_LEN = 512

In [70]:
    model = transformers.GPT2LMHeadModel.from_pretrained(MODEL_NAME)
    tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)

In [71]:
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [72]:
def break_text_to_pieces(text_path: str, tokenizer: transformers.PreTrainedTokenizer, block_len: int = 512) -> list[str]:
    """Read a file and convert it to tokenized blocks, edding <|endoftext|> to each block"""
    with open(text_path) as f:
        text = f.read()
    chunk_len0 = block_len - 1  # Leave space for a TOKEN_ENDOFTEXT
    tokens = tokenizer.encode(text)
    blocks = []
    pos = 0
    while pos < len(tokens):
        chunk = tokens[pos: pos + chunk_len0]
        chunk.append(TOKEN_ENDOFTEXT)
        blocks.append(chunk)
        pos += chunk_len0

    if len(blocks[-1]) < block_len:
        del blocks[-1]

    return blocks

In [73]:
def prepare_dsets(text_path: str, tokenizer: transformers.PreTrainedTokenizer, block_len: int):
    """Read the text, prepare the datasets """
    data = break_text_to_pieces(text_path, tokenizer, block_len)
    data_train, data_val = train_val_split(data, 0.1)
    return MyDset(data_train), MyDset(data_val)

In [74]:
def train_val_split(data: list[str], ratio: float):
    n = len(data)
    assert n >= 2
    n_val = max(1, int(n * ratio))
    return data[n_val:], data[:n_val]

In [75]:
class MyDset(torch.utils.data.Dataset):
    """A custom dataset that serves 1024-token blocks as input_ids == labels"""
    def __init__(self, data: list[list[int]]):
        self.data = []
        for d in data:
            input_ids = torch.tensor(d, dtype=torch.int64)
            attention_mask = torch.ones(len(d), dtype=torch.int64)
            self.data.append({'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': input_ids})

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        return self.data[idx]


In [77]:
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"
    dset_train, dset_val = prepare_dsets(TEXT_CORPUS, tokenizer, BLOCK_LEN)
    loader_train = torch.utils.data.DataLoader(dset_train, batch_size=1)
    loader_val = torch.utils.data.DataLoader(dset_val, batch_size=1)

In [79]:
def train_one(model: torch.nn.Module, loader: torch.utils.data.DataLoader, optimizer: torch.optim.Optimizer):
    """Standard PyTorch training, one epoch"""
    model.train()
    losses = []
    for batch in tqdm.tqdm(loader):
        for k, v in batch.items():
            batch[k] = v.to(DEVICE)
        optimizer.zero_grad()
        out = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
        # loss, logits, past_key_values
        loss = out['loss']
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

In [80]:
def val_one(model: torch.nn.Module, loader: torch.utils.data.DataLoader):
    """Standard PyTorch eval, one epoch"""
    model.eval()
    losses = []
    for batch in tqdm.tqdm(loader):
        for k, v in batch.items():
            batch[k] = v.to(DEVICE)
        with torch.no_grad():
            out = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
        # loss, logits, past_key_values
        loss = out['loss']
        losses.append(loss.item())
    return np.mean(losses)

In [81]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [87]:
wandb.init(
    project="sadGPT",
    config={
    "learning_rate": 1e-3,
    "architecture": "gpt2",
    "dataset": "https://huggingface.co/datasets/Crapp/sadQuotes/raw/main/quotes.csv",
    "epochs": 20,
    })

In [88]:
for epoch in range(20):
        loss_train = train_one(model, loader_train, optimizer)
        loss_val = val_one(model, loader_val)
        print(f'{epoch} : loss_train={loss_train}, loss_val={loss_val}')
        wandb.log({"loss_train": loss_train, "loss_val": loss_val})

100%|██████████| 14/14 [00:00<00:00, 18.60it/s]


0 : loss_train=0.11798471010945462, loss_val=5.875938756125314


100%|██████████| 14/14 [00:00<00:00, 19.19it/s]


1 : loss_train=0.11066545162487913, loss_val=5.8920718261173795


100%|██████████| 14/14 [00:00<00:00, 20.00it/s]


...

17 : loss_train=0.0944123996766629, loss_val=6.115964242390224


100%|██████████| 14/14 [00:00<00:00, 19.47it/s]


18 : loss_train=0.08316223000486692, loss_val=6.0909091745104105


100%|██████████| 14/14 [00:00<00:00, 19.69it/s]

19 : loss_train=0.08470383475500125, loss_val=6.1462615217481344


In [92]:
    if loss_val <= loss_train:
        wandb.alert(
            title='Validation lower that Train?',
            text=f'Val {loss_val} is below the Train losss {loss_train}',
        )
        print('Alert triggered')

In [93]:
wandb.finish()

loss_train,█▇▆▇▅▄▆▅▆▅▃▂▃▃▂▃▅▃▁▁
loss_val,▂▂▁▄▆▇▇▅▄▆▃▃█▄███▇▆▇
loss_train,0.0847
loss_val,6.14626


In [95]:
        model.save_pretrained('./trained_model/')
        tokenizer.save_pretrained('./trained_model/')

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [133]:
    prompt = 'Purpose of Life?'
    #prompt = 'Purpose of sex?' #memorized!
    batch = tokenizer([prompt], return_tensors='pt')
    for k, v in batch.items():
        batch[k] = v.to(DEVICE)
    out = model.generate(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], max_length=50)
    outGen=tokenizer.batch_decode(out.cpu())
    print(outGen)
    outGenFirst = '. '.join(outGen[0].split('\n')[:-1])
    print(outGenFirst.strip(prompt))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Purpose of Life?\nAre you doing it all the time\n But if you do it with involvement and pride, it’s a quality of involvement, not an act\n\nIn positive exchange, you give whatever you can without caring what']
. Are you doing it all the time.  But if you do it with involvement and pride, it’s a quality of involvement, not an act.


In [126]:
from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')
def generate(text):
    result = generator(text, max_length=300, num_return_sequences=1)
    #result = generator(text, max_length=30, num_return_sequences=1)
    return result[0]["generated_text"]

In [128]:
generate(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Purpose of Life? (1993)\n\nDangerous to Children, Not Kids(1989)\n\nDangerous To Adults, Not Children(1993)\n\nDevoted, Kind, Compassionate, Love a Lot(1991) Part 1(1992, 1993, 1994)(1993,1994)\n\nDogma Is Harmful, Loving Ourselves. (1991)\n\nDevoted, Kind, Compassionate, Love a Lot(1991) Part 2(1992)\n\nDevoted, Kind, Compassionate, Love a Lot(1991) Part 3(1992)\n (1994, 1994)\n\nDr. Lacey's Dictionary of Sex, Drugs, and Relationships(1991)\n\nDr. Lacey's Dictionary of Sex, Drugs, and Relationships(1991) Part 4(1993)\n\nDangerous to Children, Not Kids(1991)\n\nDangerous To Adults, Not Children(1993)\n\nDeeply Awkward, Kind, Compassionate, Like Love a Lot(1992)\n\nDevoted, Kind, Compassionate, Love a Lot(1992) Part 5(1992)\n\nDevoted, Kind, Compassionate, Love a Lot(1992) Part 6(1992)\n\nDeeply Awkward, Kind, Compassionate, Like Love a Lot(1992) Part 7(1992)\n\nDeeply Awkward, Kind, Compassionate"